In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import os
import csv

In [2]:
# a function to check whether CAPE is in a profile
#input: one line of data(string)
#return: boolean value

def find_cape(data):
    if "Convective Available Potential Energy: " in data:
        return True
    else:
        return False

In [3]:
# a function to strip data
#input: one line of data(string)
#return: data(string)

def data_strip(data):
    return data.strip()

In [4]:
# get the CAPE value from soup object
# Args: soup(BeautifulSoup)
# return: cape(float)

def get_cape(soup):
    cape="No value"
    cape1=soup.find_all('pre')[1].string
    split_cape=cape1.splitlines()
    strip_cape=list(map(data_strip,split_cape))
    n=list(map(find_cape,strip_cape))
    if True in n:
        t=n.index(True)
        cape_line=strip_cape[t]
        index=cape_line.index(":")
        cape=float(cape_line[index+2:])
    return cape

In [5]:
#a function to get the latitude from a profile
#input: soup(Beautifulsoup)
#return: latitude(float)

def get_latitude(soup):
    latitude='no value'
    if soup!=0:
        pre=soup.find_all('pre')
        if pre:
            cape1=soup.find_all('pre')[1].string
            cape_data=' '.join(cape1.splitlines()[3].split())
            t=''.join(cape_data.split(':',1)[:1])
            if t=='Station latitude':
                a=''.join(cape_data.split(':',1)[1:])
                latitude=float(a.replace(" ", ""))
            else:
                cape_data=' '.join(cape1.splitlines()[4].split())
                a=''.join(cape_data.split(':',1)[1:])
                latitude=float(a.replace(" ", ""))
    return latitude

In [6]:
#a function to get the longitude from a profile
#input: soup(Beautifulsoup)
#return: longitude(float)

def get_longitude(soup):
    longitude='no value'
    if soup!=0:
        pre=soup.find_all('pre')
        if pre:
            cape1=soup.find_all('pre')[1].string
            cape_data=' '.join(cape1.splitlines()[4].split())
            t=''.join(cape_data.split(':',1)[:1])
            if t=='Station longitude':
                a=''.join(cape_data.split(':',1)[1:])
                longitude=float(a.replace(" ", ""))
            else:
                cape_data=' '.join(cape1.splitlines()[5].split())
                a=''.join(cape_data.split(':',1)[1:])
                longitude=float(a.replace(" ", ""))
    return longitude

In [7]:
#open the digit code txt file
with open('digit code.txt','r') as f:
    digit_code = f.read().splitlines()
digit_code.remove('Europe')
digit_code.remove('North America')
digit_code.remove('South America')
digit_code.remove('Southeast Asia')

In [8]:
# the function to return soup object from html file
#input: year(string),month(string),day(string),code(string)
#return: soup(Beautifulsoup)

def get_soup(year,month,day,code):
    soup=0
    path='data/sounding?region=naconf&TYPE=TEXT:LIST&YEAR='+year+'&MONTH='+month+'&FROM='+day+'00&TO='+day+'00&STNM='+code
    if os.path.exists(path):
        with open(path,'rb') as f:
            a=f.read()
        soup= BeautifulSoup(a, 'html.parser')
    return soup

In [9]:
#input： one line of data(string)
#return: pressure(float)

def get_pres(data):
    return float(data[0:7])

#input： one line of data(string)
#return: temperature(float)

def get_temp(data):
    return float(data[14:21])

#input： one line of data(string)
#return: mixing ratio(float)

def get_humi(data):
    return float(data[35:42])

In [10]:
#get required variables from soup
#input: soup(Beautifulsoup),year(string),month(string),day(string),code(string)
#return: frame(Dataframe): a dataframe contains all the information required

def get_input(soup,year,month,day,code):
    frame=None
    key={}
    pre=soup.find_all('pre')
    if pre:
        cape=get_cape(soup)
        data=pre[0].string
        data_by_line=data.splitlines()
        new_data_by_line=data_by_line[5:]
        if len(new_data_by_line)>=20: 
            split_data_new=list(filter(lambda x: x[35:42].strip()!="" and x[0:7].strip()!="" and x[14:21].strip()!="", new_data_by_line))
            if len(split_data_new)!=0:
                pres_list=list(map(get_pres,split_data_new))
                temp_list=list(map(get_temp,split_data_new))
                humi_list=list(map(get_humi,split_data_new))
                if pres_list[-1]<=100:
                    key['code']=[code]*len(split_data_new)
                    key['year']=[year]*len(split_data_new)
                    key['month']=[month]*len(split_data_new)
                    key['day']=[day]*len(split_data_new)
                    key['pres']=pres_list
                    key['temp']=temp_list
                    key['humi']=humi_list
                    key['cape']=[cape]*len(split_data_new)
# 
                    frame = pd.DataFrame(key)
    return frame
        
    

In [11]:
# creating dates

years=['2020','2019','2018','2017']
months=['01','02','03','04','05','06','07','08','09','10','11','12']
days=[]
ddays=[]
for i in range(9):
    days.append('0'+str(i+1))
    ddays.append('0'+str(i+1))
for i in range(10):
    days.append('1'+str(i))
    ddays.append('1'+str(i))
for i in range(10):
    days.append('2'+str(i))
    ddays.append('2'+str(i))
leapdays=days.copy()
noleapdays=days.copy()
noleapdays.remove('29')
days.append('30')
days.append('31')
ddays.append('30')

In [12]:
# get all the URLs

url_list=[]
for code in digit_code:
    for year in years:
        for month in months:
            if month in ['01','03','05','07','08','10','12']:
                for day in days:
                    url='http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR='+year+'&MONTH='\
                        +month+'&FROM='+day+'00&TO='+day+'00&STNM='+code
                    data=[url,code,year,month,day]
                    url_list.append(data)
            elif month=='02':
                if year=='2020':
                    for day in leapdays:
                        url='http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR='+year+'&MONTH='\
                            +month+'&FROM='+day+'00&TO='+day+'00&STNM='+code
                        data=[url,code,year,month,day]
                        url_list.append(data)
                else:
                    for day in noleapdays:
                        url='http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR='+year+'&MONTH='\
                            +month+'&FROM='+day+'00&TO='+day+'00&STNM='+code
                        data=[url,code,year,month,day]
                        url_list.append(data)
            else:
                for day in ddays:
                    url='http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR='+year+'&MONTH='\
                            +month+'&FROM='+day+'00&TO='+day+'00&STNM='+code     
                    data=[url,code,year,month,day]
                    url_list.append(data)

In [13]:
# a function to achieve interpolation
# input: frame(Dataframe)
# return: (dataframe(Dataframe),cape(float))

def interp(frame):
    original_pressure=frame['pres']
    original_temperature=frame['temp']
    original_humidity=frame['humi']
    cape=frame['cape'][0]
    min_p=np.min(frame['pres'])
    max_p=np.max(frame['pres'])
    delta_p = (max_p-min_p) /69
    if delta_p==0:
        my_pressures=np.array([min_p]*70)
    else:
        my_pressures= np.linspace(min_p, max_p,70)
#         my_pressures=np.append(my_p,max_p)
    temp_on_my_pressures=np.interp(my_pressures, np.flip(original_pressure), np.flip(original_temperature))
    humi_on_my_pressures=np.interp(my_pressures, np.flip(original_pressure), np.flip(original_humidity))
    f={}
    f['pres']=my_pressures
    f['temp']=temp_on_my_pressures
    f['humi']=humi_on_my_pressures
    dataframe = pd.DataFrame(f)
    return (dataframe,cape)
    

In [14]:
# a loop to extract and create input data from URLs

soup_list={}
i=0
for url in url_list:
    i=i+1
    code=url[1]
    year=url[2]
    month=url[3]
    day=url[4]
    string=code+year+month+day
    soup=get_soup(year,month,day,code)
#     print(string,end='\r')
    if soup!=0:
        frame=get_input(soup,year,month,day,code)
        if frame is not None:
            if frame.empty==False:
                value=interp(frame)
                soup_list[string]=value
    print(i/len(url_list)*100,end='\r')

In [15]:
# save input data as .txt files and save CAPE into .csv file
#although this step is similar as the next cell, this step can easily access each files from the local machine

f = open('cape.csv','w',)
csv_writer = csv.writer(f)
i=0
count=0
for key,value in soup_list.items():
    i=i+1
    data=np.c_[value[0]['temp'],value[0]['pres'],value[0]['humi']]
    if value[1]!="No value"and value[1]<3000:
        np.savetxt('traindata/'+key,data,delimiter=',')
        csv_writer.writerow((key,value[1]))
        count=count+1
    print(i/len(soup_list)*100,end='\r')
f.close()

In [16]:
#store all input data into one .txt file, in order to prevent reloading all inputdata again and again
#this step just is a trick to load the file

f = csv.reader(open('cape.csv','r'))
next(f)
array=np.loadtxt('traindata/0601120200101',delimiter=',')
count=1
for e in f:
    if float(e[1])<3000:
        data=np.loadtxt('traindata/'+e[0],delimiter=',')
        count=count+1
        array=np.append(array,data)
    print(count/900,end='\r')
np.savetxt('xvalues',array)

In [19]:
# creating a dict where key is the digit code and value is the lat and lon

code_index={}
i=0
for code in digit_code:
    for month in months[0:6]:
        for day in days[0:10]:
            year='2020'
            soup=get_soup(year,month,day,code)
            if soup!=0:
                lat=get_latitude(soup)
                lon=get_longitude(soup)
                index=(lon,lat)
                if code not in code_index:
                    code_index[code]=index
                if code in code_index and code_index[code]==('no value','no value'):
                    code_index[code]=index


In [22]:
#creating position.csv file

f = open('position.csv','w',)
csv_writer = csv.writer(f)
for key,value in code_index.items():
    data=(key,value[0],value[1])
    csv_writer.writerow(data)
f.close()